Dependencies:
 - conda install numpy
 - conda install pandas
 - conda install cartopy
 - conda install libcomcat

In [ ]:
%matplotlib inline

In [7]:
# stdlib imports
from datetime import datetime
import pickle
from io import StringIO

# NEIC imports
from libcomcat.search import search, get_event_by_id
from libcomcat.dataframes import (get_summary_data_frame, 
                                  get_pager_data_frame,
                                  get_detail_data_frame)

# third party imports
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Define useful constants

In [ ]:
TIMEFMT = '%b %d, %Y'
OCEAN_COLOR = [0.77647059, 0.76862745, 0.79215686]
LAND_COLOR = [0.92156863, 0.92156863, 0.92156863]
YELLOW = np.array([255,255,10])/255
ORANGE = np.array([252,135,9])/255
RED = np.array([251,0,7])/255

# Download Large Events Since 1900

In [ ]:
minmag = 7.5
maxmag = 9.9
stime = datetime(1900,1,1)
etime = datetime.utcnow()

events = search(starttime=stime,
                endtime=etime,
               minmagnitude=minmag,
               maxmagnitude=maxmag)
bigframe = get_summary_data_frame(events)

(Load files from here if Internet goes down)

In [ ]:
bigframe = pd.read_csv('bigframe.csv')

In [ ]:
print('%i M7.5+ events since %s' % (len(bigframe), stime.strftime(TIMEFMT)))
bigframe.head()

# Make Map of Large Events

In [ ]:
# make a map
map_proj = ccrs.Robinson()
geo_proj = ccrs.PlateCarree()
fig = plt.figure(figsize=(15,12))
ax1 = plt.subplot(1, 1, 1, projection=map_proj)
ax1.stock_img()
scale = '50m'
land = cfeature.NaturalEarthFeature(category='physical',
                                    name='land',
                                    scale=scale,
                                    facecolor=LAND_COLOR)
ocean = cfeature.NaturalEarthFeature(category='physical',
                                     name='ocean',
                                     scale=scale,
                                     facecolor=OCEAN_COLOR)
#ax1.add_feature(land)
#ax1.add_feature(ocean)
ax1.coastlines(resolution=scale, zorder=10, linewidth=1)

ax1.plot(bigframe.longitude, bigframe.latitude, 'r.',transform=geo_proj);
ax1.set_title('M 7.5+ events since 1900');

# Map Two Years of Significant PAGER Alerts

In [ ]:
# get pager loss and exposure information (currently PAGER results only go back a few years)
stime = datetime(2017,6,1)
etime = datetime.utcnow()

yellows = search(starttime=stime,
               endtime=etime,
               alertlevel='yellow')
oranges = search(starttime=stime,
               endtime=etime,
               alertlevel='orange')
reds = search(starttime=stime,
               endtime=etime,
               alertlevel='red')

In [ ]:
with open('yellows.pkl','rb') as yellowfile:
    yellows = pickle.load(yellowfile)
with open('oranges.pkl','rb') as orangefile:
    oranges = pickle.load(orangefile)
with open('reds.pkl','rb') as redfile:
    reds = pickle.load(redfile)
print(len(yellows), len(oranges), len(reds))

In [ ]:
print('%i Yellow Alerts' % (len(yellows)))
print('%i Orange Alerts' % (len(oranges)))
print('%i Red Alerts' % (len(reds)))

In [ ]:
# make a map
map_proj = ccrs.Robinson()
geo_proj = ccrs.PlateCarree()
fig = plt.figure(figsize=(15,12))
ax1 = plt.subplot(1, 1, 1, projection=map_proj)
ax1.stock_img()
scale = '50m'
land = cfeature.NaturalEarthFeature(category='physical',
                                    name='land',
                                    scale=scale,
                                    facecolor=LAND_COLOR)
ocean = cfeature.NaturalEarthFeature(category='physical',
                                     name='ocean',
                                     scale=scale,
                                     facecolor=OCEAN_COLOR)
ax1.coastlines(resolution=scale, zorder=2, linewidth=1)

for event in yellows:
    ax1.plot(event.longitude, event.latitude, 
             color=YELLOW, marker='o', zorder=5,
             markersize=14, transform=geo_proj)

for event in oranges:
    ax1.plot(event.longitude, event.latitude, 
             color=ORANGE, marker='o', zorder=6,
             markersize=14, transform=geo_proj)

for event in reds:
    ax1.plot(event.longitude, event.latitude, 
             color=RED, marker='o', zorder=7,
             markersize=14, transform=geo_proj)
    
ax1.set_title('Significant events since Jul 1, 2017');

# Map of DYFI Responses

In [13]:
detail = get_event_by_id('ci38457511')
dyfi = detail.getProducts('dyfi')[0]
geo1km_bytes, url = dyfi.getContentBytes('cdi_geo_1km.txt')
geo1km = geo1km_bytes.decode('utf-8')
cols = ['utmc1', 'mmi', 'nresp', 'dist', 'lat', 'lon', 'nbad', 'error', 'utmc2']
usecols = ['lat', 'lon', 'mmi', 'nresp', 'dist', 'nbad', 'error']
df = pd.read_csv(StringIO(geo1km), 
                 skiprows=1, 
                 names=cols,
                 usecols=usecols,
                 index_col=False)
df
df.head()

,mmi,nresp,dist,lat,lon,nbad,error
0,3.4,1,690,39.5896,-123.7745,0,0.33
1,3.8,1,684,39.6536,-123.6236,0,0.33
2,1.0,1,587,38.8083,-123.0172,0,0.33
3,1.0,1,595,39.0515,-122.9133,0,0.33
4,2.0,1,549,38.4386,-122.8109,0,0.33


In [ ]:
# get exposure data for orange/red events
sigframe = get_pager_data_frame(oranges[0].getDetailEvent())
for orange in oranges[1:]:
    detail = orange.getDetailEvent();
    dframe = get_pager_data_frame(detail)
    sigframe = sigframe.append(dframe.iloc[0])
for red in reds[1:]:
    detail = red.getDetailEvent();
    dframe = get_pager_data_frame(detail)
    sigframe = sigframe.append(dframe.iloc[0])
sigframe

In [ ]:
with open('yellows.pkl','wb') as yellowfile:
    pickle.dump(yellows, yellowfile)
with open('oranges.pkl','wb') as orangefile:
    pickle.dump(oranges, orangefile)
with open('reds.pkl','wb') as redfile:
    pickle.dump(reds, redfile)

In [ ]:
del yellows
del oranges
del reds

In [ ]:
bigframe.to_csv('bigframe.csv')

In [ ]:
del bigframe